#### 📚Chapter5 자연어 처리 소개
##### 📌언어를 숫자로 인코딩하기

[토큰화 시작하기]

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
sentences = [
    'Today is a sunny day',
    'Today is a rainy day'
]

In [3]:
# 토큰화할 수 있는 단어 개수 지정
tokenizer = Tokenizer(num_words = 100)

In [4]:
# 토큰화된 단어 인덱스 만들기
tokenizer.fit_on_texts(sentences)

In [6]:
# 말뭉치 안에 있는 단어 / 인덱스 (key : value)
word_index = tokenizer.word_index
word_index

{'today': 1, 'is': 2, 'a': 3, 'day': 4, 'sunny': 5, 'rainy': 6}

In [7]:
sentences = [
    'Today is a sunny day',
    'Today is a rainy day',
    'Is it sunny today?'
]

In [8]:
tokenizer.fit_on_texts(sentences)
tokenizer.word_index

{'today': 1, 'is': 2, 'a': 3, 'day': 4, 'sunny': 5, 'rainy': 6, 'it': 7}

✔️ "today?"를 "today"로 필터링

[문장을 시퀀스로 바꾸기]

In [9]:
sentences = [
    'Today is a sunny day',
    'Today is a rainy day',
    'Is it sunny today?'
]

In [10]:
tokenizer = Tokenizer(num_words = 100)

In [11]:
tokenizer.fit_on_texts(sentences)

In [13]:
word_index = tokenizer.word_index
word_index

{'today': 1, 'is': 2, 'a': 3, 'sunny': 4, 'day': 5, 'rainy': 6, 'it': 7}

In [14]:
sequences = tokenizer.texts_to_sequences(sentences)
sequences

[[1, 2, 3, 4, 5], [1, 2, 3, 6, 5], [2, 7, 4, 1]]

In [15]:
# OOV 토큰 : 새로운 텍스트 대체
test_data = [
    'Today is a snowy day',
    'Will it be rainy tomorrow?'
]

In [16]:
test_sequences = tokenizer.texts_to_sequences(test_data)
print(word_index)
print(test_sequences)

{'today': 1, 'is': 2, 'a': 3, 'sunny': 4, 'day': 5, 'rainy': 6, 'it': 7}
[[1, 2, 3, 5], [7, 6]]


In [17]:
tokenizer = Tokenizer(num_words = 100, oov_token = '<OOV>')
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

In [19]:
test_sequences = tokenizer.texts_to_sequences(test_data)
print(word_index)
print(test_sequences)

{'<OOV>': 1, 'today': 2, 'is': 3, 'a': 4, 'sunny': 5, 'day': 6, 'rainy': 7, 'it': 8}
[[2, 3, 4, 1, 6], [1, 8, 1, 7, 1]]


In [20]:
# 패딩 : 시퀀스로 바꾸었을 때 동일한 길이로 맞추기
sentences = [
    'Today is a sunny day',
    'Today is a rainy day',
    'Is it sunny today?',
    'I really enjoyed walking in the snow today'
]

In [21]:
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)

In [22]:
sequences

[[2, 3, 4, 5, 6],
 [2, 3, 4, 7, 6],
 [3, 8, 5, 2],
 [9, 10, 11, 12, 13, 14, 15, 2]]

In [23]:
# pad_sequences 함수 사용
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [24]:
# 시작부분 0으로 패딩
padded = pad_sequences(sequences)
padded

array([[ 0,  0,  0,  2,  3,  4,  5,  6],
       [ 0,  0,  0,  2,  3,  4,  7,  6],
       [ 0,  0,  0,  0,  3,  8,  5,  2],
       [ 9, 10, 11, 12, 13, 14, 15,  2]])

In [25]:
# 끝부분 0으로 패딩
padded = pad_sequences(sequences, padding = 'post')
padded

array([[ 2,  3,  4,  5,  6,  0,  0,  0],
       [ 2,  3,  4,  7,  6,  0,  0,  0],
       [ 3,  8,  5,  2,  0,  0,  0,  0],
       [ 9, 10, 11, 12, 13, 14, 15,  2]])

In [26]:
# maxlen 매개변수 : 문장 최대 길이 지정
padded = pad_sequences(sequences, padding = 'post', maxlen=6)
padded

array([[ 2,  3,  4,  5,  6,  0],
       [ 2,  3,  4,  7,  6,  0],
       [ 3,  8,  5,  2,  0,  0],
       [11, 12, 13, 14, 15,  2]])

✔️ 가장 긴 문장의 일부 단어 손실, 시작부분 잘림

In [27]:
# truncating 매개변수 : 시작 위치 정하기
padded = pad_sequences(sequences, padding='post', maxlen=6, truncating='post')
padded

array([[ 2,  3,  4,  5,  6,  0],
       [ 2,  3,  4,  7,  6,  0],
       [ 3,  8,  5,  2,  0,  0],
       [ 9, 10, 11, 12, 13, 14]])

##### 📌불용어 제거와 텍스트 정제

In [34]:
# <br> 같은 HTML 태그 제거
from bs4 import BeautifulSoup
# soup = BeautifulSoup(sentence)
# sentence = soup.get_text()

In [35]:
# 불용어
stopwords = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at",
             "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do",
             "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having",
             "he", "hed", "hes", "her", "here", "heres", "hers", "herself", "him", "himself", "his", "how",
             "hows", "i", "id", "ill", "im", "ive", "if", "in", "into", "is", "it", "its", "itself",
             "lets", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought",
             "our", "ours", "ourselves", "out", "over", "own", "same", "she", "shed", "shell", "shes", "should",
             "so", "some", "such", "than", "that", "thats", "the", "their", "theirs", "them", "themselves", "then",
             "there", "theres", "these", "they", "theyd", "theyll", "theyre", "theyve", "this", "those", "through",
             "to", "too", "under", "until", "up", "very", "was", "we", "wed", "well", "were", "weve", "were",
             "what", "whats", "when", "whens", "where", "wheres", "which", "while", "who", "whos", "whom", "why",
             "whys", "with", "would", "you", "youd", "youll", "youre", "youve", "your", "yours", "yourself",
             "yourselves"]